In [1]:
import pandas as pd


In [32]:
project_id="is3107-news"

TableId_FilePath_dict={
    "Company":"../Company/Company.csv",
    "FinanceSituation":"../Finance_situation/Finance_situation.csv",
    "StockData":"../stock_price/stock_price.csv",
    "News":"../company_news/news2.csv",
}

# file path after transfering process
# News and FinanceSituation are transfered
TableId_FilePath_dict_trans={
    "Company":"../Company/Company.csv",
    "FinanceSituation":"../Finance_situation/Finance_situation_trans.csv",
    "StockData":"../stock_price/stock_price.csv",
    "News":"../company_news/news2_trans.csv",
}

In [33]:
#remove the index column in stock_price.csv
df_stock_price=pd.read_csv(TableId_FilePath_dict["StockData"],usecols=["Ticker","Date","log_return"])
df_stock_price.to_csv(TableId_FilePath_dict_trans["StockData"],index=False)

In [34]:
# news csv
# filter chinese character
df_news=pd.read_csv(TableId_FilePath_dict["News"])

def contains_chinese(text):
    for char in text:
        if char>='\u4e00' and char<='\u9fff':
            return True
    return False

df_filter_title=df_news[~df_news["title"].apply(contains_chinese)]
df_filter_text=df_news[~df_news["text"].apply(contains_chinese)]
df_filter_text.to_csv("test.csv",index=False)


In [35]:
#time stamp to data time
df_news["publish_date"]=pd.to_datetime(df_news["publish_date"])
df_news["publish_date"]=df_news["publish_date"].dt.date
df_news.to_csv(TableId_FilePath_dict_trans["News"],index=False)

In [36]:
#finance_situation.csv
df_fin=pd.read_csv(TableId_FilePath_dict["FinanceSituation"])
df_fin["debtToEquity"]/=100
df_fin["debtToEquity"].fillna(0,inplace=True)
df_fin.to_csv(TableId_FilePath_dict_trans["FinanceSituation"],index=False)

In [17]:
from google.cloud import bigquery

client = bigquery.Client()

dataset_id = 'raw_data'
dataset_ref = client.dataset(dataset_id)

In [21]:
tables = client.list_tables(dataset_ref)
for table in tables:
    print("Table ID: {}".format(table.table_id))
    for schema_field in table.schema:
        print("Column Name: {}".format(schema_field.name))
        print("Data Type: {}".format(schema_field.field_type))
        print()

KeyboardInterrupt: 

In [13]:
def load_data(table_id,file_path):
    # this is autodectect schema mode
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True,
    )

    # specified schema
    # job_config = bigquery.LoadJobConfig(
    #     schema=[
    #         bigquery.SchemaField("id", "STRING", mode="REQUIRED", description="The Unique id of the news."),
    #         bigquery.SchemaField("title", "STRING", mode="REQUIRED", description="The title of the news."),
    #         bigquery.SchemaField("text", "STRING", mode="REQUIRED", description="The text of the news."),
    #         bigquery.SchemaField("url", "STRING", mode="NULLABLE", description="The URL of the news's source."),
    #         bigquery.SchemaField("publish_date", "DATE", mode="REQUIRED", description="The publish date of the news."),
    #         bigquery.SchemaField("author", "STRING", mode="NULLABLE", description="The author of the news."),
    #         bigquery.SchemaField("language", "STRING", mode="NULLABLE", description="The language of the news."),
    #         bigquery.SchemaField("source_country", "STRING", mode="NULLABLE", description="The country of the news's source."),
    #         bigquery.SchemaField("sentiment", "FLOAT", mode="NULLABLE", description="The sentiment score of the news."),
    #     ],
    #     autodetect=False
    # )
    
    with open(file_path, "rb") as source_file:
        job = client.load_table_from_file(source_file, table_id, job_config=job_config)

    job.result()  # Waits for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

In [24]:
for table_id,file_path in TableId_FilePath_dict_trans.items():
    print(f"load data: {table_id}:")
    load_data(f"{project_id}.{dataset_id}.{table_id}",file_path)    

load data: Company:
Loaded 368 rows and 5 columns to is3107-news.raw_data.Company
load data: FinanceSituation:
Loaded 368 rows and 12 columns to is3107-news.raw_data.FinanceSituation
load data: StockData:
Loaded 22387 rows and 4 columns to is3107-news.raw_data.StockData
load data: News:
Loaded 36900 rows and 11 columns to is3107-news.raw_data.News


In [23]:
#delete table

# for table_id,file_path in TableId_FilePath_dict.items():
#     full_table_id = f"{project_id}.{dataset_id}.{table_id}"
#     client.delete_table(full_table_id)